In [22]:
import torch
import torch.nn as nn
from pysmt.shortcuts import Symbol, And, Equals, Real, GT, Max, get_model
from pysmt.typing import REAL

In [23]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()             
        self.linear = nn.Linear(2, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):   
        x = self.linear(x)
        x = self.relu(x)            
        return x[0]

In [24]:
def train(model, all_data, labels, criterion, optimizer, num_epochs):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        num_correct = 0
    
        for (data, label) in zip(all_data, labels): 
            optimizer.zero_grad()
        
            output = model(data)
            prediction = 1 if output > 0 else 0

            loss = criterion(output, label)
            num_correct += (label == prediction)
        
            running_loss += loss.item()
            loss.backward()
            optimizer.step()

        loss = running_loss/len(labels)
        accuracy = num_correct/len(labels)
    
        print("epoch: " + str(epoch) + ", accuracy: " + str(accuracy.item()) + ", loss: " + str(loss))
        

def test(model, all_data, labels, criterion): 
    running_loss = 0.0
    num_correct = 0
    
    for (data, label) in zip(all_data, labels): 
        output = model(data)
        prediction = 1 if output > 0 else 0

        loss = criterion(output, label)
        num_correct += (label == prediction)
        
        running_loss += loss.item()

    loss = running_loss/len(labels)
    accuracy = num_correct/len(labels)
    
    print("testing accuracy: " + str(accuracy.item()) + " and loss: " + str(loss))

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Model()
model.to(device)

points = torch.tensor([[-1.0, -1.0], [-1.0, 1.0], [1.0, -1.0], [1.0, 1.0]])
labels = torch.tensor([0.0, 0.0, 1.0, 1.0], requires_grad=True)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
num_epochs = 3

In [26]:
train(model, points, labels, criterion, optimizer, num_epochs)
test(model, points, labels, criterion)

epoch: 0, accuracy: 0.5, loss: 0.6931471824645996
epoch: 1, accuracy: 0.5, loss: 0.6931471824645996
epoch: 2, accuracy: 0.5, loss: 0.6931471824645996
testing accuracy: 0.5 and loss: 0.6931471824645996


In [27]:
w1 = Symbol("w1", REAL)
w2 = Symbol("w2", REAL)
w3 = Symbol("w3", REAL)

equations = []

for (point, label) in zip(points, labels):
    x = point[0].item()
    y = point[1].item()
    calculation = w1 * Real(x) + w2 * Real(y) + w3
    relu = Max(calculation, Real(0.0))
    
    if label.item() == 0.0:
        equation = Equals(relu, Real(0.0))
    else:
        equation = GT(relu, Real(0.0))
    equations.append(equation)

formula = And(equations)
print(formula)

solution = get_model(formula)
if model:
    print(solution)
    model.linear.weight.data[0][0] = float(solution[w1].constant_value())
    model.linear.weight.data[0][1] = float(solution[w2].constant_value())
    model.linear.bias.data[0] = float(solution[w3].constant_value())

else:
    print("No solution found")

(((((... + ...) <= 0.0) ? 0.0 : ((... + ...) + w3)) = 0.0) & ((((... + ...) <= 0.0) ? 0.0 : ((... + ...) + w3)) = 0.0) & (0.0 < (((... + ...) <= 0.0) ? 0.0 : ((... + ...) + w3))) & (0.0 < (((... + ...) <= 0.0) ? 0.0 : ((... + ...) + w3))))
w1 := 1/2
w3 := 0.0
w2 := 0.0


In [28]:
test(model, points, labels, criterion)

testing accuracy: 1.0 and loss: 0.5836120843887329
